NOTE : Considering some of GEE's user limits and avoiding some system errors, such as "ERROR: user memory limit exceeded","TimeoutError", etc,we will process or download some data beforehand and upload it to GEE Assets, where it will be called up when needed.

Special reminder: the following variables or parameters need to be modified

Must be amended : "roi","region","elevation"

May have to be modified (Depending on your actual situation) : "number_land","number_water","number_seasonal_water"

In [1]:
### If you are trying to use geemap in coutries where Gooogle Services are blocked (e.g., China), 
### you will need a VPN,then replace "10809" with your "proxy port number"to connect to Earth Engine servers.
### Otherwise, you might encounter a connection timeout issue.

import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
#### Initializing GEE

import geemap
import ee
Map=geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
#### Setting the boundaries of the study area.
#### Format: ee.Geometry.Rectangle(minLng, minLat, maxLng, maxLat)

roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
#### Defining variables
region = 'wuhan'

# Definition of related functions

In [ ]:
#### Removing outliers in sample set 

bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
#### cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
#### k-mean聚类
def removal_outliers(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

In [ ]:
totalSamples = ee.FeatureCollection([])
for year in range(1999,2021,5):
    print(year)
    pw_nw = ee.FeatureCollection('users/311605001111/' + region + '/' + region + '_PWL_' + str(year))
    permanent_water = pw_nw.filter(ee.Filter.eq('waterclass',1)).randomColumn('random',1,'uniform').sort('random').limit(500)
    no_water = pw_nw.filter(ee.Filter.eq('waterclass',0)).randomColumn('random',1,'uniform').sort('random').limit(3000)
    # print('Number of permanent water sampled： ',permanent_water.size().getInfo())
    # print('Number of land sampled： ',no_water.size().getInfo())
    sww = ee.FeatureCollection('users/311605001111/' + region + '/' + region + '_SW_' + str(year))
    sw = sww.randomColumn('random',1,'uniform').sort('random').limit(500)
    # print('Number of seasonal water sampled： ',sw.size().getInfo())
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    cluster_samples = ee.FeatureCollection([permanent_water,no_water]).flatten()
    clusterer = ee.Clusterer.wekaKMeans(2).train(cluster_samples,bands)
    q = ee.FeatureCollection(sw.cluster(clusterer))
    q1 = q.filter(ee.Filter.eq('cluster',1))
    q0 = q.filter(ee.Filter.eq('cluster',0))
    b1 = ee.FeatureCollection(q1).filter(ee.Filter.lt('AWEI',0)).size()
    b2 = ee.FeatureCollection(q1).filter(ee.Filter.gt('AWEI',0)).size()
    c1 = ee.FeatureCollection(q0).filter(ee.Filter.lt('AWEI',0)).size()
    c2 = ee.FeatureCollection(q0).filter(ee.Filter.gt('AWEI',0)).size()
    qq1 = ee.Algorithms.If(ee.Number(b1).lt(ee.Number(b2)),q1.map(lambda i:i.set({'waterclass':1})),q1.map(lambda i:i.set({'waterclass':0})))
    qq0 = ee.Algorithms.If(ee.Number(c1).lt(ee.Number(c2)),q0.map(lambda i:i.set({'waterclass':1})),q0.map(lambda i:i.set({'waterclass':0})))
    seasonWater = ee.FeatureCollection([qq1,qq0]).flatten()
    total_Samples = ee.FeatureCollection([permanent_water,no_water,seasonWater]).flatten()
    # print("Number of remaining samples before filtering: ", total_Samples.size().getInfo())
    filtered_samples = removal_outliers(total_Samples)
    # print("Number of remaining samples after filtering: ",filtered_samples.size().getInfo())
    totalSamples = totalSamples.merge(filtered_samples)
    print("Total sample： ",totalSamples.size().getInfo())

In [ ]:
#### The final sample set is stored in GEE's Assets
## "dataset_id" indicates the file name; "assetID" indicates the file path in GEE's Assets.
## They need to be modified to suit your situation

dataset_id = region + '_9920'
assetID = 'users/311605001111/' + region + '/' + dataset_id
task = ee.batch.Export.table.toAsset(**{
    'collection': totalSamples,
    'description': dataset_id,
    'assetId': assetID
})
task.start()